In [ ]:
import os
import openai
import time
#to be set up under your openai account
openai.organization = ""
openai.api_key = ""
#openai.Model.list()

In [ ]:
#we currently used gpt-3.5-turbo, we could try gpt4 too.
#temperature = 1
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=1):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    time.sleep(0.2)
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [ ]:
# persona from PersonaChat
from random import shuffle
file_name = 'data/ConvAI2/test_self_original.txt'
cand_list = {}
max_line=None
# Read the file and split into lines
persona = []
dial = []
lock = 0
index_dial = 0
data = {}
count=0
with open(file_name) as fin:
    for i, line in enumerate(fin):
        #print(line)
        #print('')
        line=line.strip()
        nid, line = line.split(' ', 1)          
        if(int(nid)==1 and lock==1):
            #print(str(sorted(persona)))
            count+=1
            sorted_per = str(sorted(persona))
            if(sorted_per in data):
                data[sorted_per].append(dial)
            else:
                data[sorted_per] = [dial]
            persona = []
            dial = []
            lock = 0
            index_dial = 0
        lock = 1
        if '\t' in line:
            u, r, _, cand  = line.split('\t')   
            cand = cand.split('|')
            shuffle(cand)
            for c in cand:
                if c in cand_list:
                    pass
                else:
                    cand_list[c] = 1 
            dial.append({"nid":index_dial,"u":u,"r":r})
            index_dial += 1
        else:
            r=line.split(":")[1][1:-1]
            persona.append(str(r))
        i+=1

In [ ]:
print('persona number:',count)
persona_first_count = [eval(k) for k, v in data.items()]

print(data[str(sorted(persona_first_count[0]))][0])
print('. '.join(sorted(persona_first_count[0])))

In [ ]:
def MessageConstruct(Persona, PersonaChatData):
    messages = []
    for i in range(len(Persona)):
        for utt_list in PersonaChatData[str(sorted(Persona[i]))]:
            for idx in range(len(utt_list)):
                m = []
                m_prompt = {}
                m_prompt['role'] = 'system'
                Persona_single = '. '.join(sorted(Persona[i]))
                # prompt to be explored
                Prompt = f'You are an open-domain chatbot talking like a human. Users are able to chat with you on different topics. \
                No chatbot-like talks such as "I am a chatbot", "I live in a server", "as a chatbot..." or similar expressions. \
                Make sure you talk like a human with human thinking logics.\
                You are assigned with a specific persona. \
                You need to talk like you are the assigned persona. \
                Make sure to use specific details from the provided persona when you talk.\
                The persona is delimited by triple backticks. ``` {Persona_single} ```'
                m_prompt['content'] = Prompt
                m.append(m_prompt)
                for j in range(0, idx+1):
                    utt_j = utt_list[j]
                    m_dialogue_u = {}
                    m_dialogue_u['role'] = 'user'
                    m_dialogue_u['content'] = utt_j['u']
                    m.append(m_dialogue_u)
                    if j != idx:
                        m_dialogue_a = {}
                        m_dialogue_a['role'] = 'assistant'
                        m_dialogue_a['content'] = utt_j['r']
                        m.append(m_dialogue_a)
                messages.append(m)
    return messages

def MessageConstruct_withExpectedOutput(Persona, PersonaChatData):
    messages = []
    for i in range(len(Persona)):
        for utt_list in PersonaChatData[str(sorted(Persona[i]))]:
            for idx in range(len(utt_list)):
                m = []
                m_prompt = {}
                m_prompt['role'] = 'system'
                #print('. '.join(sorted(Persona[i])))
                Persona_single = '. '.join(sorted(Persona[i]))
                # prompt to be explored
                Prompt = f'You are an open-domain chatbot talking like a human. Users are able to chat with you on different topics. \
                No chatbot-like talks such as "I am a chatbot", "I live in a server", "as a chatbot..." or similar expressions. \
                Make sure you talk like a human with human thinking logics.\
                You are assigned with a specific persona. \
                You need to talk like you are the assigned persona. \
                Make sure to use specific details from the provided persona when you talk.\
                The persona is delimited by triple backticks. ``` {Persona_single} ```'
                m_prompt['content'] = Prompt
                m.append(m_prompt)
                for j in range(0, idx+1):
                    utt_j = utt_list[j]
                    m_dialogue_u = {}
                    m_dialogue_u['role'] = 'user'
                    m_dialogue_u['content'] = utt_j['u']
                    m.append(m_dialogue_u)
                    m_dialogue_a = {}
                    m_dialogue_a['role'] = 'assistant'
                    m_dialogue_a['content'] = utt_j['r']
                    m.append(m_dialogue_a)
                messages.append(m)
    return messages

In [ ]:
requests_list = MessageConstruct(persona_first_count, data)
requests_list_withexpectedoutput = MessageConstruct_withExpectedOutput(persona_first_count, data)

In [ ]:
requests_list[0]

In [ ]:
from tqdm.notebook import tqdm
results = []
count = -1

for times in range(3):
    print("---------------------------------")
    print("---------------------------------")
    for i in tqdm(range(10)): # use 10 examples to evalute your prompt
        count += 1
        if count < 14468:
            continue
        result_peritem = {}
        #prompt to be explored
        result_peritem['prompt'] = 'You are an open-domain chatbot talking like a human. Users are able to chat with you on different topics. \
                    No chatbot-like talks such as "I am a chatbot", "I live in a server", "as a chatbot..." or similar expressions. \
                    Make sure you talk like a human with human thinking logics.\
                    You are assigned with a specific persona. \
                    You need to talk like you are the assigned persona. \
                    Make sure to use specific details from the provided persona when you talk.\
                    The persona is delimited by triple backticks. ``` {Persona_single} ```'
        result_peritem['persona'] = requests_list[i][0]['content'].split("``` ")[1]
        result_peritem['conversation_history'] = requests_list[i][1:]
        result_peritem['expected_output'] = requests_list_withexpectedoutput[i][-1]
        response_from_ChatGPT = get_completion_from_messages(requests_list[i], temperature=1)
        result_peritem['ChatGPT_output'] = response_from_ChatGPT
        results.append(result_peritem)
        

        print(response_from_ChatGPT)
    


In [ ]:
import csv

def save_dict_to_csv(dict_list, file_path):
    keys = dict_list[0].keys()
    with open(file_path, 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(dict_list)
        
save_dict_to_csv(results, 'results_prompt.csv')

In [ ]:
len(results)